<a href="https://colab.research.google.com/github/SolKidonakis/AA2TP2/blob/main/Ejercicio%202/AA2TP2EJ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TRABAJO PRACTICO N°2: APRENDIZAJE AUTOMATICO 2**

**ALUMNA: SOL KIDONAKIS**

**LIBRERIAS**

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout

In [2]:
# Configurar la GPU si está disponible
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Install tensorflow 2.15
!pip install tensorflow==2.15.0

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.15.0


**CARGA DEL DATASET**

In [3]:
# Descargar el dataset
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
)

# Leer el contenido
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Longitud del texto: {len(text)} caracteres")


Longitud del texto: 1115394 caracteres


In [4]:
# Mostrar las primeras líneas
print(text[:500])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
vocab = sorted(set(text))
print(f'{len(vocab)} caracteres únicos en el texto')


65 caracteres únicos en el texto


**PREPROCESAMIENTO PARA EL MODELO CARACTER A CARACTER**

Vectorización del texto

In [5]:

# Crear la capa para convertir caracteres a IDs
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

# Crear la capa inversa para convertir IDs a caracteres
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

# Función para convertir texto desde IDs
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


Dividir el texto en secuencias de entrada y objetivo


In [6]:
# Convertir el texto a IDs
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Dividir en secuencias de longitud deseada
seq_length = 100
examples_per_epoch = len(text) // seq_length
sequences = tf.data.Dataset.from_tensor_slices(all_ids).batch(seq_length+1, drop_remainder=True)

# Función para dividir en entrada y objetivo
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Crear dataset final
dataset = sequences.map(split_input_target)

# Crear batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)


**DEFINICION MODELO CARACTER A CARACTER**

In [7]:
# Definir el modelo
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        return (x, states) if return_state else x

# Parámetros del modelo
vocab_size = len(ids_from_chars.get_vocabulary())
embedding_dim = 256
rnn_units = 1024

# Crear el modelo
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)


**ENTRENAMIENTO DEL MODELO**

In [8]:
# Función de pérdida
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# Compilar el modelo
model.compile(optimizer='adam', loss=loss)

# Entrenar el modelo
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/10
172/172 [==============================] - 868s 5s/step - loss: 2.7566
Epoch 2/10
172/172 [==============================] - 869s 5s/step - loss: 2.0159
Epoch 3/10
172/172 [==============================] - 868s 5s/step - loss: 1.7431
Epoch 4/10
172/172 [==============================] - 869s 5s/step - loss: 1.5761
Epoch 5/10
172/172 [==============================] - 871s 5s/step - loss: 1.4714
Epoch 6/10
172/172 [==============================] - 864s 5s/step - loss: 1.4000
Epoch 7/10
172/172 [==============================] - 880s 5s/step - loss: 1.3462
Epoch 8/10
172/172 [==============================] - 889s 5s/step - loss: 1.2996
Epoch 9/10
172/172 [==============================] - 863s 5s/step - loss: 1.2586
Epoch 10/10
172/172 [==============================] - 861s 5s/step - loss: 1.2193


**GENERACION DE TEXTO**

In [11]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        predicted_logits = predicted_logits[:, -1, :] / self.temperature
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states


In [14]:
# Fragmentos iniciales
start_strings = [
    "ROMEO: What light through yonder window breaks?",
    "JULIET: O Romeo, Romeo! wherefore art thou Romeo?",
    "HAMLET: To be or not to be, that is the question.",
    "MACBETH: Is this a dagger which I see before me?",
    "KING LEAR: Blow, winds, and crack your cheeks! Rage!"
]


In [23]:
def generate_for_start_strings(start_strings, temperatures=[0.5, 1.0, 1.5], sequence_lengths=[10, 20, 50], num_chars=300):
    """
    Genera texto para fragmentos iniciales con distintas temperaturas y longitudes.

    Args:
        start_strings (list[str]): Lista de fragmentos iniciales.
        temperatures (list[float]): Temperaturas a probar.
        sequence_lengths (list[int]): Longitudes de los fragmentos iniciales.
        num_chars (int): Número de caracteres a generar.

    Returns:
        dict: Resultados organizados por fragmento, temperatura y longitud.
    """
    results = {}

    for start_string in start_strings:
        results[start_string] = {}

        for temp in temperatures:
            one_step_model.temperature = temp
            results[start_string][f"Temperature {temp}"] = {}

            for length in sequence_lengths:
                truncated_start = start_string[:length]
                states = None
                next_char = tf.constant([truncated_start])
                result = [next_char]

                # Generar texto
                for _ in range(num_chars):
                    next_char, states = one_step_model.generate_one_step(next_char, states=states)
                    result.append(next_char)

                # Guardar el texto generado
                generated_text = tf.strings.join(result)[0].numpy().decode('utf-8')
                results[start_string][f"Temperature {temp}"][f"Length {length}"] = generated_text

    return results


In [24]:

temperatures = [0.5, 1.0, 1.5]  # Diferentes valores de temperatura
sequence_lengths = [10, 20, 50]  # Diferentes longitudes iniciales
num_chars = 300  # Número de caracteres a generar


results = generate_for_start_strings(start_strings, temperatures=temperatures, sequence_lengths=sequence_lengths, num_chars=num_chars)


In [25]:
# Mostrar los resultados generados
for start_string, temp_results in results.items():
    print(f"\n### Fragmento inicial: {start_string} ###")
    for temp, length_results in temp_results.items():
        print(f"\n--- {temp} ---")
        for length, text in length_results.items():
            print(f"\nLongitud inicial {length}:\n{text[:500]}")  # Mostrar los primeros 500 caracteres



### Fragmento inicial: ROMEO: What light through yonder window breaks? ###

--- Temperature 0.5 ---

Longitud inicial Length 10:
ROMEO: What is the part?

ROMEO:
The more I shall not see him that you have as death.

DUCHESS OF YORK:
Why, he shall die, the triumphs of the deep was state-bed,
And here is sweat as you, the world death is come to go.

BENVOLIO:
The foot are such a maid we heard of men,
The senate arms against the death, th

Longitud inicial Length 20:
ROMEO: What light the children less the heart of season
That I mean to see the measure of her very trimm
That you should beget the first hath seld me in:
And let him seek to go about her love.
O, we must need to me what I less you,
That we were deadly looks on him: the house
What counsel shall dear love, who are sent f

Longitud inicial Length 50:
ROMEO: What light through yonder window breaks?

SICINIUS:
Well, sir, you shall not say
When I am not been since to curse your head,
And trust my heart as the deep from men
That sh

Temperatura 0.5: Ideal para coherencia estilística, pero menos creativa.
Temperatura 1.0: Mejor equilibrio entre coherencia y creatividad.
Temperatura 1.5: Más creativo, pero menos confiable.
Longitud inicial:

Longitudes mayores (20-50): Generan texto más relacionado con el fragmento inicial y mejoran la coherencia temática.

Para resultados más fieles al estilo de Shakespeare, es mejor usar temperatura 1.0 y longitudes iniciales entre 20-50 caracteres.
Para experimentos más creativos, usar temperatura 1.5, aunque puede perder coherencia.

**PREPROCESAMIENTO PARA EL MODELO PALABRA A PALABRA**

In [9]:
import re

# Tokenizar el texto en palabras
words = re.findall(r'\b\w+\b', text.lower())
word2idx = {u: i for i, u in enumerate(set(words))}
idx2word = np.array(list(set(words)))

# Convertir las palabras a índices
text_as_int = np.array([word2idx[w] for w in words])

# Crear secuencias de entrada y salida
seq_length = 20
examples_per_epoch = len(words) // seq_length

word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = word_dataset.batch(seq_length + 1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Crear lotes de entrenamiento
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

**DEFINICION DEL MODELO**

In [11]:
# Construcción del modelo
embedding_dim = 256
rnn_units = 1024
vocab_size_words = len(word2idx)

model_words = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size_words, embedding_dim, input_length=seq_length),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size_words)
])

model_words.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Entrenamiento
EPOCHS = 15
history = model_words.fit(dataset, epochs=EPOCHS)

Epoch 1/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 207s 652ms/step - loss: 7.4637
Epoch 2/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 197s 635ms/step - loss: 6.2363
Epoch 3/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 196s 632ms/step - loss: 5.8436
Epoch 4/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 195s 630ms/step - loss: 5.5181
Epoch 5/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 195s 630ms/step - loss: 5.1788
Epoch 6/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 196s 631ms/step - loss: 4.7812
Epoch 7/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 196s 632ms/step - loss: 4.3166
Epoch 8/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 197s 637ms/step - loss: 3.8603
Epoch 9/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 213s 688ms/step - loss: 3.4358
Epoch 10/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 226s 727ms/step - loss: 3.0498
Epoch 11/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 216s 694ms/step - loss: 2.6994
Epoch 12/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 202s 651ms/step - loss: 2.3756
Epoch 13/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 201s 648ms/step - loss: 2.0919
Epoch 14/15
310/310 ━━━━━━━━━━━━━━━━━━━━ 198s 640ms/step - loss: 1.8381
E

**GENERACION DE TEXTO**

In [21]:
def generate_for_start_strings_words(
    model, word2idx, idx2word, start_strings, temperatures=[0.5, 1.0, 1.5], sequence_lengths=[5, 10, 15], num_generate=50, seq_length=20
):
    """
    Genera texto para fragmentos iniciales con distintas temperaturas y longitudes iniciales.

    Args:
        model: Modelo entrenado.
        word2idx (dict): Diccionario palabra -> índice.
        idx2word (dict): Diccionario índice -> palabra.
        start_strings (list[str]): Lista de fragmentos iniciales.
        temperatures (list[float]): Temperaturas a probar.
        sequence_lengths (list[int]): Longitudes de los fragmentos iniciales.
        num_generate (int): Número de palabras a generar.
        seq_length (int): Longitud máxima de la secuencia de entrada.

    Returns:
        dict: Resultados organizados por fragmento inicial, temperatura y longitud.
    """
    results = {}

    for start_string in start_strings:
        results[start_string] = {}

        for temp in temperatures:
            results[start_string][f"Temperature {temp}"] = {}

            for length in sequence_lengths:
                # Truncar el fragmento inicial a la longitud deseada
                truncated_start = " ".join(start_string.split()[:length])

                # Convertir el texto inicial en índices
                input_eval = [word2idx.get(word, word2idx.get('<UNK>', 0)) for word in truncated_start.split()]
                input_eval = tf.expand_dims(input_eval[-seq_length:], 0)  # Ajustar longitud al tamaño esperado

                # Lista para almacenar las palabras generadas
                text_generated = []

                for _ in range(num_generate):
                    predictions = model(input_eval)
                    predictions = tf.squeeze(predictions, 0)  # Elimina la dimensión del batch
                    predictions /= temp  # Ajusta por temperatura

                    # Seleccionar la siguiente palabra
                    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
                    predicted_word = idx2word[predicted_id]
                    text_generated.append(predicted_word)

                    # Actualizar la entrada para el modelo
                    input_eval = tf.expand_dims(input_eval.numpy().tolist()[0][-seq_length + 1:] + [predicted_id], 0)

                # Guardar el texto generado
                generated_text = " ".join(truncated_start.split() + text_generated)
                results[start_string][f"Temperature {temp}"][f"Length {length}"] = generated_text

    return results


In [22]:

# Generar texto probando distintas temperaturas y longitudes iniciales
results_words = generate_for_start_strings_words(
    model=model_words,
    word2idx=word2idx,
    idx2word=idx2word,
    start_strings=start_strings,
    temperatures=[0.5, 1.0, 1.5],
    sequence_lengths=[5, 10, 15],
    num_generate=50,
    seq_length=20
)

# Mostrar resultados
for start_string, temp_results in results_words.items():
    print(f"\n### Fragmento inicial: {start_string} ###")
    for temp, length_results in temp_results.items():
        print(f"\n--- {temp} ---")
        for length, text in length_results.items():
            print(f"\nLongitud inicial {length}:\n{text[:500]}")  # Mostrar los primeros 500 caracteres



### Fragmento inicial: ROMEO: What light through yonder window breaks? ###

--- Temperature 0.5 ---

Longitud inicial Length 5:
ROMEO: What light through yonder isabella i warrant it for it hath more honest and not too much i ll warrant him as for his life were he and in him he should have more than at the prison angelo for this good lord he was no power to do him such a good person

Longitud inicial Length 10:
ROMEO: What light through yonder window breaks? o the good he s and so you will you master of men s master and his mistress made i pray you let him see hortensio i know it is as she tranio but as well i tender your princely babe and the moon that it doth now have no

Longitud inicial Length 15:
ROMEO: What light through yonder window breaks? o the warrant i ll not be it clown he comes not so he s a kind and would it were in my heart petruchio why think it as i of a kind an o er i am sure that e er i saw a man i should saw my

--- Temperature 1.0 ---

Longitud inicial Length 5:


Coherencia Estilística:

El modelo captura bien el formato de diálogos y utiliza un vocabulario acorde al estilo de Shakespeare.

Efecto de la Temperatura:
0.5 (baja): Genera texto predecible y repetitivo con frases más coherentes pero menos creativas.
1.0 (media): Balance entre creatividad y coherencia; introduce más variedad en el texto.
1.5 (alta): Texto más creativo pero con menor coherencia y frases más desordenadas.
Influencia de la Longitud Inicial:

Secuencias cortas: Menos conexión con el inicio, más propenso a incoherencias.
Secuencias largas: Mejor relación temática con el inicio, mayor coherencia.
Limitaciones:
En temperaturas altas y secuencias largas, el texto pierde sentido lógico y repite estructuras.

**CONCLUSION FINAL**

Carácter a Carácter:
Este enfoque es ideal para capturar el estilo único de Shakespeare, con rimas, ritmos y construcciones poéticas. Es altamente creativo, pudiendo inventar palabras y estructuras estilísticamente plausibles, pero tiende a ser menos coherente semánticamente.

Palabra a Palabra:
Más adecuado para generar texto lógico y temáticamente coherente. Trabaja mejor con relaciones contextuales entre palabras, pero tiene menor creatividad estilística y está limitado al vocabulario aprendido.